<a href="https://colab.research.google.com/github/samko5sam/programming-language-class/blob/main/0320_DataVisualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [3]:
import pandas as pd
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1IkFP9_-C0sdyZNgp3gjeoqPyv3MV2fQqUFKgTKRLF2w/edit?usp=sharing')

HW2

In [5]:
# 從 Google Sheets 讀取資料
worksheet = gsheets.get_worksheet(2)
data = worksheet.get_all_values()
df = pd.DataFrame(data)
df.columns=df.iloc[0]
df = df.iloc[1:]
# 將數值欄位轉換為數字類型
numeric_columns = ['開市', '最高', '最低', '收市', '調整後的收市價', '成交量']

df['成交量'] = df['成交量'].str.replace(',', '')
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')  # 轉為數值，無效數據變為 NaN
df['日期'] = pd.to_datetime(df['日期'], format='%Y年%m月%d日')

df

,日期,開市,最高,最低,收市,調整後的收市價,成交量
1,2025-04-09,453.49,502.06,453.16,499.10,499.10,28499321.0
2,2025-04-08,479.68,482.49,449.60,456.74,456.74,19002700.0
3,2025-04-07,449.54,480.76,442.80,463.56,463.56,31285300.0
4,2025-04-04,481.28,483.30,464.24,465.52,465.52,30966500.0
5,2025-04-03,501.00,503.66,493.50,494.16,494.16,16226000.0
...,...,...,...,...,...,...,...
248,2024-04-16,464.05,465.51,461.59,462.78,456.66,6761400.0
249,2024-04-15,473.47,473.59,462.85,463.61,457.48,9530000.0
250,2024-04-12,472.77,474.05,467.91,469.57,463.36,6954300.0
251,2024-04-11,473.98,477.43,470.67,476.06,469.76,5109900.0


In [11]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# 1. 收盤價的時間序列折線圖
fig1 = px.line(df, x='日期', y='收市', title='VOO ETF 收盤價趨勢')
fig1.update_layout(xaxis_title='日期', yaxis_title='收盤價 (美元)')
fig1.show()

收盤價趨勢：

* 圖表目的：看出股價變化的趨勢。

* 趨勢分析：
    * 在這段時間內，VOO ETF 呈現明顯的下跌趨勢，從最高點 560 美元跌至 460 美元，總跌幅約 17.9%。
* 關鍵轉折點：
    * 3 月下旬至 4 月初：斷崖式下跌，與美國關稅政策有關。
* 投資建議：
    * 短期投資者：3 月下旬至 4 月初這段期間的高波動性和下跌趨勢表明市場風險較高，短期交易需謹慎。
    * 長期投資者：VOO 作為標普 500 指數 ETF，長期趨勢通常與美國經濟表現相關。雖然短期下跌顯著，但如果相信美國經濟長期增長潛力，當前價格可能提供一個較低的買入機會。

In [14]:
# 計算日收益率
df['日收益率'] = df['調整後的收市價'].pct_change() * 100

# 2. 移動平均線與布林帶圖
# 計算 20 日移動平均線和布林帶
df['20日均線'] = df['調整後的收市價'].rolling(window=20).mean()
df['標準差'] = df['調整後的收市價'].rolling(window=20).std()
df['布林上軌'] = df['20日均線'] + 2 * df['標準差']
df['布林下軌'] = df['20日均線'] - 2 * df['標準差']

fig1 = go.Figure()
# 收盤價
fig1.add_trace(go.Scatter(x=df['日期'], y=df['調整後的收市價'], mode='lines', name='調整後收盤價'))
# 20 日均線
fig1.add_trace(go.Scatter(x=df['日期'], y=df['20日均線'], mode='lines', name='20日均線', line=dict(color='orange')))
# 布林帶
fig1.add_trace(go.Scatter(x=df['日期'], y=df['布林上軌'], mode='lines', name='布林上軌', line=dict(color='gray', dash='dash')))
fig1.add_trace(go.Scatter(x=df['日期'], y=df['布林下軌'], mode='lines', name='布林下軌', line=dict(color='gray', dash='dash'), fill='tonexty', fillcolor='rgba(200, 200, 200, 0.2)'))
fig1.update_layout(title='VOO ETF 移動平均線與布林帶', xaxis_title='日期', yaxis_title='價格 (美元)')
fig1.show()

<ipython-input-14-82a1f407b5a6>:2: FutureWarning:

The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



移動平均線與布林帶圖
* 圖表目的：結合 20 日移動平均線和布林帶，分析價格趨勢和波動性，識別潛在的超買或超賣區域。
* 趨勢分析：
    * 移動平均線：20 日均線可以平滑短期波動，顯示價格的中期趨勢。如果價格持續低於 20 日均線（如 2025 年 3 月至 4 月），表明趨勢偏空；若價格突破均線，可能預示反彈。
    * 布林帶：布林帶上下軌表示價格的波動範圍（均線 ± 2 倍標準差）。價格接近或突破上軌可能表示超買，接近或跌破下軌可能表示超賣。
根據之前的折線圖和蠟燭圖，VOO 在 2025 年 4 月初跌至 440 美元，若此時價格跌破布林下軌，可能表明市場超賣，短期內可能出現反彈。
* 投資建議：

    價格在去年 11 月初跌破布林下軌且隨後反彈，可能是一個短期買入機會。
    若價格持續低於 20 日均線且布林帶收窄，可能表示該時間段波動較低，市場可能在等待新東西出現。

In [15]:
# 3. 成交量加權價格變動圖
# 計算價格變動（收盤價 - 開盤價）
df['價格變動'] = df['收市'] - df['開市']
# 成交量加權價格變動
df['成交量加權變動'] = df['價格變動'] * df['成交量']
fig3 = px.scatter(df, x='日期', y='成交量加權變動', size='成交量', color='價格變動',
                  title='VOO ETF 成交量加權價格變動',
                  labels={'成交量加權變動': '成交量加權價格變動 (美元*成交量)'},
                  color_continuous_scale='RdYlGn')
fig3.update_layout(xaxis_title='日期', yaxis_title='成交量加權價格變動')
fig3.show()

成交量加權價格變動圖
* 圖表目的：通過成交量加權的價格變動，探索交易活躍度與價格波動的關係，識別異常交易行為。
* 趨勢分析：
    * 圖表中每個點表示一天的成交量加權價格變動（價格變動 × 成交量），點的大小表示成交量，顏色表示價格變動方向（紅色為下跌，綠色為上漲）。
    * 下跌期間（如 4 月 2 日至 4 月 6 日）出現大成交量且價格變動為負（紅色大點），表明賣壓極強，可能是受美影響恐慌性拋售。
* 投資建議：

    若後續成交量減少且價格變動趨於平穩，可能表明賣壓減弱，市場進入底部盤整階段（下跌後進入一個低位區間，波動幅度變小且成交量減少，市場情緒趨於平穩，買賣雙方力量平衡，等待新的趨勢方向。）。

    投資者可關注成交量加權變動的異常點，這些點可能與重大新聞或市場轉折相關，值得進一步調查。

課堂練習

In [ ]:
# 從 Google Sheets 讀取資料
worksheet = gsheets.get_worksheet(0)
data = worksheet.get_all_values()
df = pd.DataFrame(data)
df.columns=df.iloc[0]
df = df.iloc[1:]
# 將數值欄位轉換為數字類型
numeric_columns = ['除權息年度', '股利', '除息前股價', '填息天數', '年股利', '年殖利率(%)']
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')  # 轉為數值，無效數據變為 NaN
df

,除權息年度,股利,除息日,發放日,除息前股價,填息天數,年股利,年殖利率(%)
1,2025,2.70,2025/1/17,2025/2/20,198.05,4,2.70,1.36
2,2024,1.00,2024/7/16,2024/8/9,196.70,95,4.00,2.44
3,2024,3.00,2024/1/17,2024/2/21,131.65,3,4.00,2.44
4,2023,1.90,2023/7/18,2023/8/11,132.00,127,4.50,3.60
5,2023,2.60,2023/1/30,2023/3/7,118.10,2,4.50,3.60
6,2022,1.80,2022/7/18,2022/8/19,115.50,4,5.00,3.78
7,2022,3.20,2022/1/21,2022/3/4,149.10,4,5.00,3.78
8,2021,0.35,2021/7/21,2021/8/24,137.20,2,3.40,2.43
9,2021,3.05,2021/1/22,2021/3/9,143.00,27,3.40,2.43
10,2020,0.70,2020/7/21,2020/8/24,97.05,2,3.60,3.81


In [ ]:
import plotly.express as px

# 按年度分組並取平均值
df_grouped = df.groupby('除權息年度')[['年股利', '年殖利率(%)']].mean().reset_index()

# 創建折線圖
fig = px.line(df_grouped,
              x='除權息年度',
              y=['年股利', '年殖利率(%)'],
              title='年度股利與殖利率變化',
              labels={'value': '金額/百分比', '除權息年度': '年度', 'variable': '指標'},
              height=600,
              width=800)

# 更新折線圖，添加數據點標記
fig.update_traces(mode='lines+markers', marker=dict(size=8))  # 添加點並設置大小

# 更新佈局
fig.update_layout(
    yaxis_title="金額(元) / 百分比(%)",
    legend_title="指標",
    template="plotly_white"
)

# 顯示圖表
fig.show()

In [ ]:
# 從 Google Sheets 讀取資料
worksheet = gsheets.get_worksheet(1)
data = worksheet.get_all_values()
df = pd.DataFrame(data)
df.columns=df.iloc[0]
df = df.iloc[1:]

# 將 '日期' 欄位轉換為日期型態
df['日期'] = pd.to_datetime(df['日期'])

# 將其他數值欄位轉換為浮動數型態
numeric_columns = [
    '美元╱新台幣', '人民幣╱新台幣', '歐元╱美元', '美元╱日幣',
    '英鎊╱美元', '澳幣╱美元', '美元╱港幣', '美元╱人民幣',
    '美元╱南非幣', '紐幣╱美元'
]

df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric)

df

,日期,美元╱新台幣,人民幣╱新台幣,歐元╱美元,美元╱日幣,英鎊╱美元,澳幣╱美元,美元╱港幣,美元╱人民幣,美元╱南非幣,紐幣╱美元
1,2025-02-03,32.968,4.488725,1.02220,155.615,1.22895,0.61380,7.79370,7.34465,18.96275,0.55545
2,2025-02-04,32.985,4.515515,1.03210,155.475,1.24190,0.62035,7.78945,7.30480,18.75225,0.56055
3,2025-02-05,32.838,4.516933,1.04025,153.305,1.24995,0.62730,7.78850,7.27000,18.63240,0.56755
4,2025-02-06,32.826,4.501725,1.03725,152.575,1.24615,0.62585,7.78665,7.29185,18.67180,0.56560
5,2025-02-07,32.754,4.492997,1.03920,151.775,1.24465,0.62915,7.78940,7.29000,18.42690,0.56825
6,2025-02-10,32.839,4.489058,1.03115,152.325,1.23990,0.62685,7.78865,7.31535,18.50900,0.56510
7,2025-02-11,32.835,4.492518,1.03045,152.035,1.23475,0.62825,7.79225,7.30880,18.47655,0.56515
8,2025-02-12,32.861,4.493249,1.03785,153.645,1.24630,0.62905,7.78995,7.31340,18.45175,0.56535
9,2025-02-13,32.802,4.495514,1.04185,154.315,1.24920,0.62730,7.78960,7.29660,18.48685,0.56405
10,2025-02-14,32.770,4.508792,1.04740,152.595,1.25790,0.63355,7.78495,7.26800,18.39775,0.56960


In [ ]:
import plotly.express as px

fig = px.line(df, x="日期", y="美元╱新台幣", title="美元/新台幣匯率走勢")
fig.show()